In [23]:
## cd into project folder
%cd '/content/drive/MyDrive/dev/DSPS23/tutorial/task2'

/content/drive/MyDrive/dev/DSPS23/tutorial/task2


In [5]:
# !pip install -U --no-cache-dir gdown --pre

In [7]:
# download training data
# !gdown --id '1iU0qE7sM_AvdQnh0emkbcwwh2d3pE5TG'

In [9]:
# extract zipped training data
# !unzip 'task2_data.zip' -d './'

In [10]:
import os, cv2, shutil
import pandas as pd 

In [12]:
## prepare data for yolo model
def prep_data(csv_file, images):

  names = { 1:'longitudinal_high',2:'longitudinal_low',3:'longitudinal_medium', 
           4:'grass', 5:'patch_high', 6:'manhole_high', 7:'transverse_high',
           8:'transverse_low', 9:'transverse_medium', 10:'diag_high', 
           11:'diag_low', 12:'diag_medium', 13:'alligator_high', 
           14:'alligator_low', 15:'alligator_medium',  16:'block_low', 
           17:'block_high', 18:'block_medium'}


  dst_imgs_path = 'training/images'
  dst_lbls_path = 'training/labels'
  if not os.path.isdir('training/images'):
    os.makedirs(dst_imgs_path)
    os.makedirs(dst_lbls_path)
  
  df = pd.read_csv(csv_file)
  img_ids = df['img_name'].unique()
  for img in img_ids:
    df_cur = df[df['img_name'] == img]
    full_img_path = os.path.join(images,img)
    frame = cv2.imread(full_img_path)
    height,width,chan = frame.shape
    for data in df_cur.values.tolist():
        cimg,x1,y1,x2,y2 = data[0:-2]
        ccolor = data[-2] + '_' + data[-1]
        value = [i for i in names if names[i]==ccolor]
        cls = value[0]
        dw = 1. / width
        dh = 1. / height
        x = (x1 + x2) / 2.0
        y = (y1 + y2) / 2.0
        w = x2 - x1
        h = y2 - y1
        x = x * dw
        w = w * dw
        y = y * dh
        h = h * dh

        file_path_img = os.path.join(dst_lbls_path, img)
        filename, file_extension = os.path.splitext(file_path_img)
        # print (filename,file_extension)
        with open(file_path_img.replace(file_extension,'.txt'), 'a+') as f:
          f.write(' '.join([str(int(cls)), str(float(x)), str(float(y)), str(float(w)), str(float(h))]))
          f.write('\n')

    shutil.copy(full_img_path,os.path.join(dst_imgs_path, img))

  # print (df.head())

In [13]:
## converts annotations into yolo format
## creates new folder - training data
csv_file = 'task2_data/annotations.csv'
images = 'task2_data/images'

prep_data(csv_file, images)

In [24]:
!git clone 'https://github.com/UM-Titan/DSPS23.git'

Cloning into 'DSPS23'...
remote: Enumerating objects: 236, done.
remote: Counting objects: 100% (236/236), done.
remote: Compressing objects: 100% (184/184), done.
remote: Total 236 (delta 61), reused 183 (delta 38), pack-reused 0
Receiving objects: 100% (236/236), 13.95 MiB | 16.11 MiB/s, done.
Resolving deltas: 100% (61/61), done.


In [26]:
%cd 'DSPS23'

/content/drive/MyDrive/dev/DSPS23/tutorial/task2/DSPS23


In [19]:
# !pip install -r 'requirements.txt'

In [27]:
!python train_dsps.py --data data/dsps2.yaml --epochs 100 --weights yolov5s.pt --cfg yolov5s.yaml  --batch-size 64

train_dsps: weights=yolov5s.pt, cfg=yolov5s.yaml, data=data/dsps2.yaml, hyp=data/hyps/hyp.scratch-low.yaml, epochs=100, batch_size=64, imgsz=640, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, bucket=, cache=None, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=runs/train, name=exp, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, seed=0, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
remote: Enumerating objects: 15070, done.
remote: Counting objects: 100% (69/69), done.
remote: Compressing objects: 100% (40/40), done.
remote: Total 15070 (delta 32), reused 60 (delta 29), pack-reused 15001
Receiving objects: 100% (15070/15070), 14.10 MiB | 16.00 MiB/s, done.
Resolving deltas: 100% (10323/10323), done.
From https://github.com/ultralytics/yolov5
 * [new branch]      add/

In [31]:
## Download test data
# !gdown --id '11iKT_JlR5OPhLLdMtP1wkxNQ-fz6_who'
# !unzip 'test.zip' -d './'

In [33]:
!python detect_dsps2.py --weights runs/train/exp/weights/best.pt --source images/

detect_dsps2: weights=['runs/train/exp/weights/best.pt'], source=images/, data=data/coco128.yaml, imgsz=[640, 640], conf_thres=0.25, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=False, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=runs/detect, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False, vid_stride=1
YOLOv5 🚀 14a4958 Python-3.8.10 torch-1.13.1+cu116 CUDA:0 (Tesla T4, 15110MiB)

Fusing layers... 
YOLOv5s summary: 157 layers, 7058671 parameters, 0 gradients, 15.9 GFLOPs
Speed: 0.5ms pre-process, 9.5ms inference, 1.0ms NMS per image at shape (1, 3, 640, 640)
Results saved to runs/detect/exp


In [ ]:
## submit the 'submission_task2_final.json' created in the current folder
